In [2]:
%matplotlib inline

In [4]:
# Ensure python 3 forward compatibility
# 保证python3前向兼容
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import theano
# By convention, the tensor submodule is load as T
# 为了方便使用，张量子模型被加载为T
import theano.tensor as T

# 基础
## Symbolic variables符号变量
在Theano中，所有的算法都是符号化定义的。这种方法更类似书写数学表达式。下列的Theano变量都是符号化的；它们都没有明确的值。

In [10]:
# theano.tensor子模块有很多自带的符号化变量类型
# 在这里，我们定义一个标量（scalar）（0-d）变量
# 在参数中给出变量的名字
foo = T.scalar('foo')
# 现在，我们可以定义另一个变量bar，它的值是foo的平方
bar = foo**2
# 这也是一个theano变量
print(type(bar))
print(bar.type)
# 使用theano的pp（pretty print）函数，我们可以看到
# bar是由foo的平方符号化定义的
print(theano.pp(bar))

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, scalar)
(foo ** TensorConstant{2})


## 函数
为了使用Theano进行实际的计算，你需要定义符号函数，它可以使用实际值调用并返回一共真实值。

In [12]:
# 我们还不能使用foo和bar进行计算
# 我们需要先定义theano函数
# theano.function中的第一个参数定义了函数的输入
# 需要注意bar依赖于foo，所以foo是这个函数的一共输入
# 在给定foo的值后，theano.function会编译代码来计算bar的值
f = theano.function([foo], bar)
print(f(3))

9.0


In [14]:
# 替代性的，在一些情况下你可以使用一个符号变量的估值方法
# 这种方法比定义一个函数更方便
# 这个估值方法接受一个字典，其中键是theano的变量，值是这些变量的值
print(bar.eval({foo: 3}))

9.0


In [15]:
# 我们也可以使用Python函数来构建Theano变量
# 这看起来很麻烦，但是可以让复杂例子的语法更清晰
def square(x):
    return x**2
bar = square(foo)
print(bar.eval({foo: 3}))

9.0


## theano.tensor
Theano也有向量、矩阵、张量的变量类型。theano.tensor子模块有很多函数可以操作这些变量。

In [16]:
A = T.matrix('A')
x = T.vector('x')
b = T.vector('b')
y = T.dot(A, x) + b
# 注意对一个矩阵做平方操作是element-wise
z = T.sum(A**2)
# theano.function可以同时进行很多算
# 你也可以设定参数的默认值
# 我们会在后续涉及到theano.config.floatX
b_default = np.array([0, 0], dtype=theano.config.floatX)
linear_mix = theano.function([A, x, theano.Param(b, default=b_default)], [y, z])
# 为A, x, b设定值
print(linear_mix(np.array([[1, 2, 3],
                           [4, 5, 6]], dtype=theano.config.floatX), #A
                 np.array([1, 2, 3], dtype=theano.config.floatX), #x
                 np.array([4, 5], dtype=theano.config.floatX))) #b
# 使用b的默认值
print(linear_mix(np.array([[1, 2, 3],
                           [4, 5, 6]], dtype=theano.config.floatX), #A
                 np.array([1, 2, 3], dtype=theano.config.floatX))) #x

[array([ 18.,  37.]), array(91.0)]
[array([ 14.,  32.]), array(91.0)]


## Shared variables共享变量
共享变量有一些不同，它们实际上有一个明确的值，并且可以被获取和设置，同时在跨函数使用时共享这些值。它们是很方便的，因为它们在跨函数调用的时候有声明。

In [21]:
shared_var = theano.shared(np.array([[1, 2], [3, 4]], dtype=theano.config.floatX))
# 共享变量的数据类型可以由它的初始化过程推导出
print(shared_var.type())

<TensorType(float64, matrix)>


In [22]:
# 我们可以使用set_value设置一个共享变量的值
shared_var.set_value(np.array([[3, 4], [2, 1]], dtype=theano.config.floatX))
# 并且使用get_value获取值
print(shared_var.get_value())

[[ 3.  4.]
 [ 2.  1.]]


In [23]:
shared_squared = shared_var**2
# theano.function的第一个参数（inputs）告诉Theano编译函数的参数应该是怎么样的
# 由于shared_var是共享的，它已经有了值，所以它没有必要作为一个函数的输入
# 因此，当使用shared_squared时，Theano隐式地将shared_var作为一个函数的输入
# 我们不需要在theano.function的输入变量中包含它
function_1 = theano.function([], shared_squared)
print(function_1())

[[  9.  16.]
 [  4.   1.]]
